In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from utils import *

import seaborn as sns
sns.set()

## Statistical significance

Often we will want to make a distinction between two hypotheses based on data, we want to know whether one hypothesis is more likely than another based on the data we have. 

A statistical hypothesis test usually compares the oberved results with those that would occur under the null hypothesis 

In [ ]:
continuous1 = add_nan(continuous_data_one())
display(continuous1.describe())
display(continuous1.notnull().sum())

In [ ]:
bins = np.linspace(-20,20,25)
ax = sns.distplot(np.random.standard_cauchy(1000), bins=bins, kde=True)
ax.axis([-20,20,0,0.15])

In [ ]:
stats.ttest_ind(np.random.standard_cauchy(100), continuous_data_one(mean=0, size=100))

In [ ]:
d1 = add_nan(continuous_data_one(mean=0))
d2 = add_nan(np.random.standard_cauchy(1000)) + 0.5*d1
df = pd.DataFrame(data=[d1, d2])
df.T.corr()

## Statistical Tests

There are several flavors of significance testing. These all have a couple of things in common. 